In [20]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot
import sklearn.model_selection

from tensorflow.keras import backend as K

In [12]:
num_classes = 2
n = 10   # number of time steps

pattern = np.zeros((num_classes, n))

# sawtooth
x = []
y = 0.0
for i in range(n):
  x.append(y)
  if i < n/2:
    y += 0.2
  else:
    y -= 0.2
pattern[0] = x

# sin wave
pattern[1] = np.sin(np.linspace(0, np.pi*2, n))*0.5 + 0.5

instances_per_class = 150
m = num_classes * instances_per_class

X = np.zeros((m, n))
y = np.zeros(m)
for i in range(num_classes):
  for j in range(instances_per_class):
    X[i*instances_per_class+j] = pattern[i] + np.random.normal(scale=0.05, size=n)
    y[i*instances_per_class+j] = i

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.3, random_state=0)


In [13]:
def get_rnn(sequence_length=10):
  inputs = keras.layers.Input(shape=(sequence_length,1))
  x = keras.layers.SimpleRNN(1)(inputs)
  x = keras.layers.Dense(1, activation='sigmoid')(x)
  model = keras.models.Model(inputs, x)
  return model

model = get_rnn(10)
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 10, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (None, 1)              │             3 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5 (20.00 B)

 Trainable params: 5 (20.00 B)

 Non-trainable params: 0 (0.00 B)

In [21]:
K.clear_session()
with tf.device('/CPU:0'):

  model = get_rnn(10)
  model.compile(loss='binary_crossentropy', metrics=['accuracy'])
  model.fit(X_train, y_train, epochs=500,
    batch_size=16, validation_split=0.3)


Epoch 1/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4124 - loss: 0.8014 - val_accuracy: 0.5079 - val_loss: 0.7408
Epoch 2/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4711 - loss: 0.7611 - val_accuracy: 0.5079 - val_loss: 0.7381
Epoch 3/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4884 - loss: 0.7478 - val_accuracy: 0.5079 - val_loss: 0.7367
Epoch 4/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5099 - loss: 0.7340 - val_accuracy: 0.5079 - val_loss: 0.7350
Epoch 5/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4672 - loss: 0.7556 - val_accuracy: 0.5079 - val_loss: 0.7329
Epoch 6/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5203 - loss: 0.7251 - val_accuracy: 0.5079 - val_loss: 0.7314
Epoch 7/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4747 - loss: 0.7460 - val_accuracy: 0.5079 - val_loss: 0.7299
Epoch 8/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4396 - loss: 0.7615 - val_accuracy: 0.5079 - 

In [24]:
rnn_weights   = model.get_layer('simple_rnn').get_weights()
dense_weights = model.get_layer('dense').get_weights()

print(rnn_weights)
print(dense_weights)

[array([[2.8181767]], dtype=float32), array([[-0.28796187]], dtype=float32), array([-0.8274807], dtype=float32)]
[array([[2.5519626]], dtype=float32), array([-0.43602818], dtype=float32)]
